In [95]:
from pydantic import BaseModel
from typing import List, Dict
from datetime import datetime, timedelta
from pathlib import Path
from uuid import UUID, uuid4
import json
import os

# 定义文件路径
ROOMS_FILE = "./room_data.json"
BOOKINGS_FILE = "./booking_records.json"

# 定义房间模型
class Room(BaseModel):
    room_id: str
    capacity: int
    bookings: List[List[str]]  # 预定状态，包含开始和结束时间
    facilities: List[str]       # 房间设施

# 定义预订记录模型
class Booking(BaseModel):
    booking_id: str
    room_id: str
    user_name: str
    start_time: str
    end_time: str
    created_at: str

# 初始化房间数据并写入JSON文件
def initialize_rooms_data():
    """初始化房间数据并写入JSON文件"""
    rooms = {
        "R101": Room(
            room_id="R101",
            capacity=8,
            bookings=[["2023-10-01 09:00", "2023-10-01 10:30"], ["2023-10-01 14:00", "2023-10-01 15:00"]],
            facilities=["Whiteboard", "Projector"]
        ),
        "R102": Room(
            room_id="R102",
            capacity=15,
            bookings=[["2023-10-01 11:00", "2023-10-01 12:30"]],
            facilities=["Whiteboard", "Video Conference", "Teleconference"]
        ),
        "R103": Room(
            room_id="R103",
            capacity=25,
            bookings=[["2023-10-01 13:00", "2023-10-01 16:00"]],
            facilities=["Projector", "Video Conference", "Teleconference", "Whiteboard"]
        ),
        "R104": Room(
            room_id="R104",
            capacity=50,
            bookings=[["2023-10-01 10:00", "2023-10-01 12:00"], ["2023-10-01 15:30", "2023-10-01 17:00"]],
            facilities=["Projector", "Video Conference", "Teleconference", "Whiteboard", "Sound System"]
        ),
        "R105": Room(
            room_id="R105",
            capacity=4,
            bookings=[],
            facilities=["Whiteboard"]
        ),
        "R106": Room(
            room_id="R106",
            capacity=10,
            bookings=[["2023-10-01 09:30", "2023-10-01 11:30"], ["2023-10-01 13:30", "2023-10-01 14:30"]],
            facilities=["Projector"]
        ),
        "R107": Room(
            room_id="R107",
            capacity=20,
            bookings=[["2023-10-01 09:00", "2023-10-01 17:00"]], # 全天预订
            facilities=["Whiteboard", "Projector", "Video Conference"]
        ),
        "R108": Room(
            room_id="R108",
            capacity=30,
            bookings=[],
            facilities=["Whiteboard", "Projector", "Video Conference", "Teleconference"]
        ),
        "R109": Room(
            room_id="R109",
            capacity=6,
            bookings=[["2023-10-01 10:00", "2023-10-01 11:00"], ["2023-10-01 14:00", "2023-10-01 16:00"]],
            facilities=["Whiteboard"]
        ),
        "R110": Room(
            room_id="R110",
            capacity=12,
            bookings=[["2023-10-01 09:00", "2023-10-01 10:00"], ["2023-10-01 11:00", "2023-10-01 12:00"]],
            facilities=["Projector", "Whiteboard"]
        )
    }
    
    # 将房间数据转换为可序列化的字典
    rooms_dict = {room_id: room.model_dump() for room_id, room in rooms.items()}
    
    # 写入JSON文件
    with open(ROOMS_FILE, 'w', encoding="utf-8") as f:
        json.dump(rooms_dict, f, indent=4, ensure_ascii=False)
    
    return rooms_dict

# 初始化预订记录文件
def initialize_bookings_file():
    """初始化预订记录文件"""
    with open(BOOKINGS_FILE, 'w') as f:
        json.dump({}, f, indent=4, ensure_ascii=False)

# 读取房间数据
def load_rooms_data():
    """从JSON文件读取房间数据"""
    if not os.path.exists(ROOMS_FILE):
        return initialize_rooms_data()
    
    with open(ROOMS_FILE, 'r') as f:
        rooms_dict = json.load(f)
    
    # 将字典转换为Room对象
    rooms = {room_id: Room(**room_data) for room_id, room_data in rooms_dict.items()}
    return rooms

# 读取预订记录
def load_bookings():
    """从JSON文件读取预订记录"""
    if not os.path.exists(BOOKINGS_FILE):
        initialize_bookings_file()
        return {}
    
    with open(BOOKINGS_FILE, 'r') as f:
        bookings = json.load(f)
    bookings = {booking_id: Booking(**booking_data) for booking_id, booking_data in bookings.items()}
    
    return bookings

# 保存预订记录
def save_booking(booking: Booking):
    """保存预订记录到JSON文件"""
    bookings = load_bookings()
    
    # 将新的预订记录添加到字典中
    bookings[str(booking.booking_id)] = booking
    bookings_dict = {book_id: book.model_dump() for book_id, book in bookings.items()}

    # 保存到文件
    with open(BOOKINGS_FILE, 'w') as f:
        json.dump(bookings_dict, f, indent=4, ensure_ascii=False)

# 查询所有房间状态
def get_all_rooms_status(current_time) -> Dict[str, Dict]:
    """
    查询所有房间的当前预定状态。
    返回一个字典，包含每个房间的ID、未来的预定状态。
    """
    rooms = load_rooms_data()
    status = {}
    for room_id, info in rooms.items():
        status[room_id] = {
            "bookings": [b for b in info.bookings if b[0] > current_time],
        }
    return status

# 查询房间具体信息
def get_room_info(room_id: str) -> Dict:
    """
    查询指定房间的详细信息（不包括预定信息）。
    :param room_id: 房间ID
    :return: 房间的其他属性，若房间不存在则返回None
    """
    rooms = load_rooms_data()
    if room_id in rooms:
        room_info = rooms[room_id].model_dump()  # 转换为字典
        del room_info["bookings"]  # 不返回预定信息
        return room_info
    return "不存在该房间"

def get_available_rooms(start_time: str, end_time: str) -> List[str]:
    """
    查询在指定时间段内可预订的房间。
    :param start_time: 预定开始时间
    :param end_time: 预定结束时间
    :return: 可预订房间的ID列表
    """
    rooms = load_rooms_data()
    available_rooms = []
    
    # 将输入的时间字符串转换为datetime对象
    start_dt = datetime.strptime(start_time, "%Y-%m-%d %H:%M")
    end_dt = datetime.strptime(end_time, "%Y-%m-%d %H:%M")
    
    for room_id, info in rooms.items():
        is_available = True
        for booking in info.bookings:
            # 将预订时间字符串转换为datetime对象
            booking_start = datetime.strptime(booking[0], "%Y-%m-%d %H:%M")
            booking_end = datetime.strptime(booking[1], "%Y-%m-%d %H:%M")
            
            # 检查时间冲突
            if not (end_dt <= booking_start or start_dt >= booking_end):
                is_available = False
                break
        if is_available:
            available_rooms.append(room_id)
    return available_rooms

# 预定房间
def book_room(room_id: str, start_time: str, end_time: str, user_name: str) -> str:
    """
    预定指定房间。
    :param room_id: 房间ID
    :param start_time: 预定开始时间
    :param end_time: 预定结束时间
    :param user_name: 用户名
    :return: 预定成功返回成功信息，失败返回失败信息
    """
    rooms = load_rooms_data()
    if room_id in rooms:
        # 将输入的时间字符串转换为datetime对象
        start_dt = datetime.strptime(start_time, "%Y-%m-%d %H:%M")
        end_dt = datetime.strptime(end_time, "%Y-%m-%d %H:%M")
        
        # 检查时间是否可用
        for booking in rooms[room_id].bookings:
            # 将预订时间字符串转换为datetime对象
            booking_start = datetime.strptime(booking[0], "%Y-%m-%d %H:%M")
            booking_end = datetime.strptime(booking[1], "%Y-%m-%d %H:%M")
            
            # 检查时间冲突
            if not (end_dt <= booking_start or start_dt >= booking_end):
                return "该时间段房间已被预订"
        
        # 创建预订记录
        booking_id = str(uuid4())
        booking = Booking(
            booking_id=booking_id,
            room_id=room_id,
            user_name=user_name,
            start_time=start_time,
            end_time=end_time,
            created_at=datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        )
        
        
        # 保存预订记录
        save_booking(booking)
        
        return f"成功预定房间{room_id}，预订ID: {booking_id}"
    else:
        return "不存在该房间"

# 初始化程序时，确保房间数据文件和预订记录文件存在
if __name__ == "__main__":
    # if not os.path.exists(ROOMS_FILE):
    initialize_rooms_data()
    # if not os.path.exists(BOOKINGS_FILE):
    initialize_bookings_file()


q1 = "预定当前时间，一个任意的房间"
q2 = "预定一个指定的房间的一个长时间段中的2h（需要先查询然后再预定）"
q3 = "预定一个具有某个设施的房间"
q4 = "挑选一个时间，可以同时预定5个房间"
q5 = "预定一个已经被占用了的房间（如果有这个名字的预定则不得分）"
q6 = "预定一个某个时间段满足100人的房间"
q7 = "为不同的3个人预定3个不同的房间"
q8 = "预定某个时间段里面具有投影仪的最大房间"
q9 = "预定某个时间段的所有满足某个条件的房间"
q10 = "预定一个不存在的房间(如果有这个名字的预定则不得分)"

In [96]:
# 检查是否完成
def check(conditions):
    bookings = load_bookings()
    rooms = load_rooms_data()
    for booking_id, booking_data in bookings.items():
        results = [c(booking_data) for c in conditions]
        r = all(results)
        if r:
            return 1
    return 0

# 初始化数据防止干扰
initialize_rooms_data()
# if not os.path.exists(BOOKINGS_FILE):
initialize_bookings_file()

# 测试题1
def eval1():
    name = "Eric001"
    time = "2023-10-01 09:37"
    q1 = f"你在系统中的名字是{name},现在是{time}，请你预定一个现在立刻可以使用2h的会议室"
    def check_time(b):
        start_time = b.start_time
        end_time = b.end_time
        booking_start = datetime.strptime(start_time, "%Y-%m-%d %H:%M")
        booking_end = datetime.strptime(end_time, "%Y-%m-%d %H:%M")
        difference =  booking_end - booking_start
        if difference.seconds == 7200:
            return True
        else:
            return False
    conditions = [lambda b: b.user_name==name, lambda b:b.start_time == time, check_time]
    return q1, conditions
# 获取测试题和验证条件
q, conditions = eval1()
# 完成符合测试要求的动作
book_room("R103","2023-10-01 09:37", "2023-10-01 11:37", "Eric001")
print(check(conditions))

1


In [99]:
# 初始化数据防止干扰
initialize_rooms_data()
initialize_bookings_file()


# 测试题1：预定当前时间，一个任意的房间
def eval1():
    name = "Alice001"
    time = "2023-10-01 10:00"
    q1 = f"你在系统中的名字是{name},现在是{time}，请你预定一个任意的房间，从现在开始使用1小时"
    conditions = [
        lambda b: b.user_name == name,
        lambda b: b.start_time == time,
        lambda b: datetime.strptime(b.end_time, "%Y-%m-%d %H:%M") - datetime.strptime(b.start_time, "%Y-%m-%d %H:%M") == timedelta(hours=1)
    ]
    return q1, conditions

q, conditions = eval1()

# 动作
available_rooms = get_available_rooms("2023-10-01 10:00", "2023-10-01 11:00")
if available_rooms:
    result = book_room(available_rooms[0], "2023-10-01 10:00", "2023-10-01 11:00", "Alice001")
    print(result)
else:
    print("没有可用房间")

check(conditions)

成功预定房间R102，预订ID: ab724c6f-2a30-4c23-8731-cdf2f005ac98


1

In [100]:
# 测试题1：预定当前时间，一个任意的房间
def eval1():
    name = "Alice001"
    time = "2023-10-01 10:00"
    q1 = f"你在系统中的名字是{name},现在是{time}，请你预定一个任意的房间，从现在开始使用1小时"
    conditions = [
        lambda b: b.user_name == name,
        lambda b: b.start_time == time,
        lambda b: datetime.strptime(b.end_time, "%Y-%m-%d %H:%M") - datetime.strptime(b.start_time, "%Y-%m-%d %H:%M") == timedelta(hours=1)
    ]
    return q1, conditions

# 完成测试1
available_rooms = get_available_rooms("2023-10-01 10:00", "2023-10-01 11:00")
if available_rooms:
    book_room(available_rooms[0], "2023-10-01 10:00", "2023-10-01 11:00", "Alice001")
print("测试1结果:", check(eval1()[1]))

# 测试题2：预定一个指定的房间的一个长时间段中的2h
def eval2():
    name = "Bob002"
    time = "2023-10-01 13:00"
    q2 = f"你在系统中的名字是{name}，请先查询房间R108在2023-10-01的可用时间，然后预定该房间从{time}开始的2小时"
    conditions = [
        lambda b: b.user_name == name,
        lambda b: b.room_id == "R108",
        lambda b: b.start_time == time,
        lambda b: datetime.strptime(b.end_time, "%Y-%m-%d %H:%M") - datetime.strptime(b.start_time, "%Y-%m-%d %H:%M") == timedelta(hours=2)
    ]
    return q2, conditions

# 完成测试2
book_room("R108", "2023-10-01 13:00", "2023-10-01 15:00", "Bob002")
print("测试2结果:", check(eval2()[1]))

# 测试题3：预定一个具有某个设施的房间
def eval3():
    name = "Charlie003"
    time = "2023-10-01 14:00"
    q3 = f"你在系统中的名字是{name}，请预定一个具有视频会议(Video Conference)设施的房间，从{time}开始使用1.5小时"
    conditions = [
        lambda b: b.user_name == name,
        lambda b: b.start_time == time,
        lambda b: datetime.strptime(b.end_time, "%Y-%m-%d %H:%M") - datetime.strptime(b.start_time, "%Y-%m-%d %H:%M") == timedelta(minutes=90),
        lambda b: "Video Conference" in load_rooms_data()[b.room_id].facilities
    ]
    return q3, conditions

# 完成测试3
rooms = load_rooms_data()
video_conf_rooms = []
for room_id, room in rooms.items():
    if "Video Conference" in room.facilities:
        video_conf_rooms.append(room_id)
available_rooms = get_available_rooms("2023-10-01 14:00", "2023-10-01 15:30")
video_conf_available = [r for r in available_rooms if r in video_conf_rooms]
if video_conf_available:
    book_room(video_conf_available[0], "2023-10-01 14:00", "2023-10-01 15:30", "Charlie003")
print("测试3结果:", check(eval3()[1]))

# 测试题4：挑选一个时间，可以同时预定5个房间
def eval4():
    name = "David004"
    time = "2023-10-01 17:30"
    q4 = f"你在系统中的名字是{name}，请找出一个时间点，在该时间可以同时预定5个不同的房间，每个房间使用1小时"
    conditions = [
        lambda b: b.user_name == name,
        lambda b: b.start_time == time,
        lambda b: datetime.strptime(b.end_time, "%Y-%m-%d %H:%M") - datetime.strptime(b.start_time, "%Y-%m-%d %H:%M") == timedelta(hours=1)
    ]
    return q4, conditions

# 完成测试4
available_rooms = get_available_rooms("2023-10-01 17:30", "2023-10-01 18:30")
if len(available_rooms) >= 5:
    for i in range(5):
        book_room(available_rooms[i], "2023-10-01 17:30", "2023-10-01 18:30", "David004")
print("测试4结果:", check(eval4()[1]))

# 测试题5：预定一个已经被占用了的房间
def eval5():
    name = "Eve005"
    time = "2023-10-01 09:30"
    q5 = f"你在系统中的名字是{name}，请尝试预定房间R107在{time}到10:30的时间段"
    conditions = [
        lambda b: b.user_name != name or b.room_id != "R107" or b.start_time != time
    ]
    return q5, conditions

# 完成测试5
book_room("R107", "2023-10-01 09:30", "2023-10-01 10:30", "Eve005")
print("测试5结果:", check(eval5()[1]))

# 测试题6：预定一个某个时间段满足100人的房间
def eval6():
    name = "Frank006"
    time = "2023-10-01 12:00"
    q6 = f"你在系统中的名字是{name}，请预定一个可以容纳至少100人的房间，从{time}开始使用2小时"
    conditions = [
        lambda b: b.user_name == name,
        lambda b: b.start_time == time,
        lambda b: datetime.strptime(b.end_time, "%Y-%m-%d %H:%M") - datetime.strptime(b.start_time, "%Y-%m-%d %H:%M") == timedelta(hours=2),
        lambda b: load_rooms_data()[b.room_id].capacity >= 100
    ]
    return q6, conditions

# 完成测试6
rooms = load_rooms_data()
large_rooms = []
for room_id, room in rooms.items():
    if room.capacity >= 100:
        large_rooms.append(room_id)
if large_rooms:
    available_rooms = get_available_rooms("2023-10-01 12:00", "2023-10-01 14:00")
    large_available = [r for r in available_rooms if r in large_rooms]
    if large_available:
        book_room(large_available[0], "2023-10-01 12:00", "2023-10-01 14:00", "Frank006")
print("测试6结果:", check(eval6()[1]))

# 测试题7：为不同的3个人预定3个不同的房间
def eval7():
    names = ["Grace007", "Henry007", "Ivy007"]
    time = "2023-10-01 15:00"
    q7 = f"请为三个不同的人({', '.join(names)})在{time}预定三个不同的房间，每个房间使用1小时"
    conditions = [
        lambda b: b.user_name in names,
        lambda b: b.start_time == time,
        lambda b: datetime.strptime(b.end_time, "%Y-%m-%d %H:%M") - datetime.strptime(b.start_time, "%Y-%m-%d %H:%M") == timedelta(hours=1)
    ]
    return q7, conditions

# 完成测试7
names = ["Grace007", "Henry007", "Ivy007"]
available_rooms = get_available_rooms("2023-10-01 15:00", "2023-10-01 16:00")
if len(available_rooms) >= 3:
    for i in range(3):
        book_room(available_rooms[i], "2023-10-01 15:00", "2023-10-01 16:00", names[i])
print("测试7结果:", check(eval7()[1]))

# 测试题8：预定某个时间段里面具有投影仪的最大房间
def eval8():
    name = "Jack008"
    time = "2023-10-01 16:00"
    q8 = f"你在系统中的名字是{name}，请在{time}预定一个具有投影仪(Projector)设施的最大容量房间，使用1.5小时"
    
    def check_largest_projector_room(b):
        rooms = load_rooms_data()
        projector_rooms = [r for r_id, r in rooms.items() if "Projector" in r.facilities]
        if not projector_rooms:
            return False
        max_capacity_room = max(projector_rooms, key=lambda r: r.capacity)
        return b.room_id == max_capacity_room.room_id
    
    conditions = [
        lambda b: b.user_name == name,
        lambda b: b.start_time == time,
        lambda b: datetime.strptime(b.end_time, "%Y-%m-%d %H:%M") - datetime.strptime(b.start_time, "%Y-%m-%d %H:%M") == timedelta(minutes=90),
        check_largest_projector_room
    ]
    return q8, conditions

# 完成测试8
rooms = load_rooms_data()
projector_rooms = []
for room_id, room in rooms.items():
    if "Projector" in room.facilities:
        projector_rooms.append((room_id, room.capacity))
projector_rooms.sort(key=lambda x: x[1], reverse=True)
if projector_rooms:
    largest_projector_room = projector_rooms[0][0]
    available_rooms = get_available_rooms("2023-10-01 16:00", "2023-10-01 17:30")
    if largest_projector_room in available_rooms:
        book_room(largest_projector_room, "2023-10-01 16:00", "2023-10-01 17:30", "Jack008")
print("测试8结果:", check(eval8()[1]))

# 测试题9：预定某个时间段的所有满足某个条件的房间
def eval9():
    name = "Kelly009"
    time = "2023-10-01 18:00"
    q9 = f"你在系统中的名字是{name}，请预定{time}开始的1小时内所有具有白板(Whiteboard)设施且容量小于10人的房间"
    
    def check_small_whiteboard_room(b):
        rooms = load_rooms_data()
        room = rooms.get(b.room_id)
        return room and "Whiteboard" in room.facilities and room.capacity < 10
    
    conditions = [
        lambda b: b.user_name == name,
        lambda b: b.start_time == time,
        lambda b: datetime.strptime(b.end_time, "%Y-%m-%d %H:%M") - datetime.strptime(b.start_time, "%Y-%m-%d %H:%M") == timedelta(hours=1),
        check_small_whiteboard_room
    ]
    return q9, conditions

# 完成测试9
rooms = load_rooms_data()
small_whiteboard_rooms = []
for room_id, room in rooms.items():
    if "Whiteboard" in room.facilities and room.capacity < 10:
        small_whiteboard_rooms.append(room_id)
available_rooms = get_available_rooms("2023-10-01 18:00", "2023-10-01 19:00")
small_whiteboard_available = [r for r in available_rooms if r in small_whiteboard_rooms]
if small_whiteboard_available:
    for room_id in small_whiteboard_available:
        book_room(room_id, "2023-10-01 18:00", "2023-10-01 19:00", "Kelly009")
print("测试9结果:", check(eval9()[1]))

# 测试题10：预定一个不存在的房间
def eval10():
    name = "Larry010"
    time = "2023-10-01 19:00"
    q10 = f"你在系统中的名字是{name}，请尝试预定房间R999在{time}到20:00的时间段"
    conditions = [
        lambda b: b.user_name != name or b.room_id != "R999"
    ]
    return q10, conditions

# 完成测试10
book_room("R999", "2023-10-01 19:00", "2023-10-01 20:00", "Larry010")
print("测试10结果:", check(eval10()[1]))

测试1结果: 1
测试2结果: 1
测试3结果: 1
测试4结果: 1
测试5结果: 1
测试6结果: 0
测试7结果: 1
测试8结果: 0
测试9结果: 1
测试10结果: 1
